# Data Wrangling

There are standard data exploring and cleaning (section "Inspecting data") and basic data wrangling procedures (Step 1 of "Transforming Data") contained in this script. However, some heavier data transformations also happen in the "Transforming Data" section, with the most complex parts being 3 and 5. In step 3, view and complete events are allocated to offer received events. Block ids for these sets of events that identify which receive, view and complete belong together are created. In step 5, the transactions are allocated to these blocks.

## Setup

In [2]:
import pandas as pd
import numpy as np
import math
import json
import os
#% matplotlib inline
from pandasql import sqldf
from matplotlib import pyplot as plt

In [3]:
os.chdir(r'C:\Users\j\Documents\udacity-ds-nanodegree\starbucks-project')

In [13]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Inspecting data

In this section, I inspect all three dataframes to assert data quality, define steps for data cleaning, and get a first impression of the data.
Further exploration can only be carried out after complex data transformations, which will also be required to carry out the actual analysis.

In [ ]:
profile.head(5)

In [ ]:
portfolio.head(10)

In [ ]:
transcript.head(10)

### profile data

#### Summary of inspection of data in profile table:
* As I cannot judge if this is a representative sample of the Starbucks App user population, I will assume it is.
* The table only contains **unique records** of app-users. No deduplication is needed.
* Around 12.8% of these users have **missing information** for all three demographic variables. I believe that this data should not be imputed. It cannot be *derived/predicted* from any other variables, as all other variables will be part of the prediction model. In addition, these demographic values will be important predictors for the model themselves, therefore imputation like using the mean/median or mode seems too "rough" an approach. I will instead check if the people without information are systematically different from those with information, and if not, I will drop the records. In order to make this assessment, the data needs to be transformed.
* The **gender** is not 50-50, as would be expected from the general population. It is, however, possible that the split is representative of the wider Starbucks App subscriber population.
* The **age** distribution suggests that the sample is rather old, which I find surprising for Stabrucks users. Other than that the age range seems plausible.
* The  **income** appears to have been capped both at the lower and higher end. Other than that the distribution seems plausible, showing the typical right skew that income data often displays.
* The **membership** column has no missings and the data range is plausible. Most memberships are more recent, which makes sense considering that consumers probably drop out after a certain amount of time.

In [ ]:
# Number of unique users, check for duplicates
print("Number of unique user: ",profile['id'].nunique())
print("Number of users in df: ",profile.shape[0])

In [ ]:
# Assess "gender" column
print("Categorical codes for gender column: ",profile['gender'].value_counts())
print("Number of missing values in gender column: ",profile['gender'].isna().sum())

In [ ]:
# Assess "age" column distribution, code 118 is missing information
profile['age'].hist() # >> surprisingly old population considering company is Starbucks

# Recode 118 inta np.nan
profile.loc[profile['age'] == 118, 'age'] = np.nan

# get min and max values, and missing values
print("Lowest age in dataset: ",profile['age'].min())
print("Highest age in dataset: ", profile['age'].max())
print("Number of missing values in age column: ",profile['age'].isna().sum())

In [ ]:
# Assess "income" column distribution
profile['income'].hist() # >> I assume income was cropped at a min and max value

# get min and max values, and missing values
print("Lowest age in dataset: ",profile['income'].min())
print("Highest age in dataset: ", profile['income'].max())
print("Number of missing values in income column: ",profile['income'].isna().sum())

In [ ]:
# Check if the missings are the same for all three columns
all_missing = profile.loc[(profile['income'].isna()) & (profile['age'].isna()) & (profile['gender'].isna())].shape[0]
print("Records that have all three demographic attributes missing: ",all_missing)

In [ ]:
# Assess share of records without demographic information out of all records
all_missing/profile.shape[0] #>> I would lose 12.8% of the data

Need to assess if those who did not provide demographic data are systematically different from those who did to decide if I can remove them from the sample. I will compare:
* Total spend and offers completed: Those are variables that I will **predict**, so there should not be any bias in them
* offers received and type of offer received: That is the **experimental setup** and I do not want this to be biased
In order to assess this, I need to maipulate the transcipt df a bit. I need to extract amount for instance

In [ ]:
# Assess "membership" column
# Change to datetime format
'''
Source: https://datatofish.com/integers-datetime-pandas-dataframe/, Auagust 14th 2021
'How to convert Integers to datetime in Pandas DataFrame'
'''
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format = '%Y%m%d')

In [ ]:
print("Earliest membership date: ",profile['became_member_on'].min())
print("Latest membership date: ",profile['became_member_on'].max())
profile['became_member_on'].hist()
plt.show()

### portfolio data

#### Summary of inspection of data in portfolio table
* There is total of 10 offers
* Intuitively, the expectation would be that a reward rises with difficulty, and that duration rises with difficulty. 
* When examining the data through scatterplots, this relationship is not actually that clear. There are some offers that appear  less attractive than others.
* However, these offers vary on further attributes, like type and channels

In [ ]:
portfolio.head(10).sort_values(by=['offer_type'])

In [ ]:
# Check uniqueness of offer ids
portfolio['id'].nunique()

In [ ]:
# Number of offers per type
portfolio['offer_type'].hist()
plt.show()

In [ ]:
# examining relationship between attributes
plt.scatter(portfolio['reward'], portfolio['difficulty'])
plt.xlabel('reward')
plt.ylabel('difficulty')
plt.show()

In [ ]:
# examining relationship between attributes
plt.scatter(portfolio['duration'], portfolio['difficulty'])
plt.xlabel('duration')
plt.ylabel('difficulty')
plt.show()

### transaction data

#### Summary of inspection of data in transaction table
* The logical funnel for an offer is *receive >> view >> complete*
* However, this funnel can be shortened to *receive >> view* and also to *receive >> complete*, or switched to *receive >> complete >> view*
* The maximum number of offers someone can receive is 6
* Most people receive 5 offers
* Most people view 3 offers
* Most people complete 2 offers

In [ ]:
# How many offers does a person receive >> histogram
offers_h = transcript.loc[transcript['event'] == 'offer received'].groupby(['person']).size()
views_h = transcript.loc[transcript['event'] == 'offer viewed'].groupby(['person']).size()
completes_h = transcript.loc[transcript['event'] == 'offer completed'].groupby(['person']).size()

In [ ]:
# max number of offers, needed to define bins in histogram
offers_h.max()

In [ ]:
# Version with transparent bars - user version from below, just keep this code for reference. Alpha makes transparent, the lower alpha the more transparent.
plt.hist(offers_h, bins = [1,2,3,4,5,6,7],alpha = 0.3, align = 'left')
plt.hist(views_h, bins = [1,2,3,4,5,6,7], alpha = 0.3, align = 'left')
plt.hist(completes_h, bins = [1,2,3,4,5,6,7], alpha = 0.3, align = 'left')
plt.show()

In [ ]:
# Version with bars side by side
# Source Stackoverflow: https://stackoverflow.com/questions/6871201/plot-two-histograms-on-single-chart-with-matplotlib
# Question by Open the way: https://stackoverflow.com/users/171546/open-the-way
# Answer by Gustavo Bezerra: https://stackoverflow.com/users/2132753/gustavo-bezerra
plt.hist([offers_h, views_h, completes_h], bins = [1,2,3,4,5,6,7], align = 'left', label = ['received', 'viewed', 'completed'])
plt.legend()
plt.show()

## Transforming data
* **Step 1:** Wrangle all three dataframes and create master df which is based on transaction frame but without transactions. This df will be transformed to be fit for analysis.
* **Step 2:** Answer some relevant questions about the general experimental setup
* **Step 3:** Transform master df to create a table which has one record per person and offer, and which contains additional information on this particular offer.
* **Step 4:** Create derived variables
* **Step 5:** Add transaction value dataframe: matrix person on offer and no_offer and avg value.
Start with a person x hour df where each hour has a 0 (no offer) to 10, and then you can use that to lookup the transaction's value maybe? Too simple I guess, check your transaction allocation logic again.

### Step 1: Wrangling all three dataframes

In [ ]:
# profile data
'''
Source: https://datatofish.com/integers-datetime-pandas-dataframe/, Auagust 14th 2021
'How to convert Integers to datetime in Pandas DataFrame'

Source: Stackoverflow: https://stackoverflow.com/questions/53986451/calculate-date-difference-between-todays-date-and-pandas-date-series/53986547
Question by Shivam: https://stackoverflow.com/users/10851563/shivam
Answer by k88: https://stackoverflow.com/users/5682512/k88

Source: Satckoverflow: https://stackoverflow.com/questions/2119472/convert-a-timedelta-to-days-hours-and-minutes
Question by Oli: https://stackoverflow.com/users/12870/oli
Answer by Alex Martelli: https://stackoverflow.com/users/95810/alex-martelli
'''
# Calculating duration of membership
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format = '%Y%m%d')
profile['membership_duration'] = profile['became_member_on'] - pd.to_datetime("now")
profile['membership_duration'] = profile['membership_duration'].dt.days

In [ ]:
# portfolio data
# Calculating until when offer is valid
portfolio = portfolio.rename(columns={'reward':'offer_reward','channels':'offer_channels','difficulty':'offer_difficulty', 'duration':'offer_duration'})
portfolio['offer_duration'] = portfolio['offer_duration']*24
portfolio['offer_valid_until'] = np.nan


# One-hot-encoding channels
# Source Stackoverflow: https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
# Question by Melsauce: https://stackoverflow.com/users/5858873/melsauce
# Answer by piRsquared: https://stackoverflow.com/users/2336654/pirsquared
portfolio = portfolio.drop('offer_channels',1).join(portfolio['offer_channels'].str.join('|').str.get_dummies())

In [14]:
# Transaction data >> create master frame
'''
Source: Stackoverflow: https://stackoverflow.com/questions/38231591/split-explode-a-column-of-dictionaries-into-separate-columns-with-pandas
Question by Ilaffin: https://stackoverflow.com/users/6157698/llaffin
Answer by joris: https://stackoverflow.com/users/653364/joris
'''

# Turning the value column which lists different events into separate columns
master = pd.concat([transcript, pd.DataFrame(transcript['value'].tolist())], axis=1)

# There are cases where the offer id is offer_id in the 'value' dictionary. Need to combine both columns.
'''
Source: Stackoverflow: https://stackoverflow.com/questions/34989341/how-to-remove-nan-value-while-combining-two-column-in-panda-data-frame
Question by imSonuGupta: https://stackoverflow.com/users/3101669/imsonugupta
Answer by jpp: https://stackoverflow.com/users/9209546/jpp
'''
master['offer id'].update(master.pop('offer_id'))
master = master.rename(columns={'offer id':'event_offer_id','amount':'event_amount','reward':'event_reward'})

# Removing all transaction records
master = master.loc[master['event'] != 'transaction']
# Remove event_amount column as not needed
master = master.drop('event_amount', axis = 1)

In [ ]:
# Merging portfolio table
master = pd.merge(master, portfolio[['offer_type','offer_reward','offer_difficulty','offer_duration','email','web','social','mobile']], how = 'left', left_on = master['event_offer_id'], right_on = portfolio['id'])
master = master.drop('key_0', axis = 1)

# Creating derived variable offer_valid_until
master.loc[master['event'] == 'offer received', 'offer_valid_until'] = master['time'] + master['offer_duration']

### Step 2: Understand experiment setup in more detail

#### When do offers get sent out? >> At 6 distinct points in time. They are not spaced out evenly, and a slightly different amount of offers is sent out each time

In [ ]:
master_receive = master.loc[master['event'] == 'offer received']
master_receive_gr = pd.DataFrame(master_receive.groupby(['time']).size().reset_index())

In [ ]:
master_receive_gr

In [ ]:
minor_ticks = np.arange(0,600,24)
plt.scatter(master_receive_gr['time'],master_receive_gr[0])
plt.grid(axis = 'x')
plt.show()

#### Does a specific offer only get sent out at one point in time? >> No
#### Do different offers get sent out at the same time? >> Yes

In [ ]:
pd.crosstab(master_receive['event_offer_id'],master_receive['time'])

#### Can a person receive the same offer multiple times? >> Yes, the maximum a person has received the same offer is 5 times

In [ ]:
pd.crosstab(master_receive['person'], master_receive['event_offer_id']).max().max()

#### Can a person receive a new offer whilst another one is still active? >> Yes, this happens frequently

In [ ]:
# Creating subset
offer_logic_check = master_receive[['person','event_offer_id','time','offer_valid_until']].sort_values(by=['person', 'time'])

In [ ]:
 # shifting row values one row down
offer_logic_check['shift_time'] = offer_logic_check['offer_valid_until'].shift(periods = 1)
offer_logic_check['shift_person'] = offer_logic_check['person'].shift(periods = 1)
# Create checking column, populate only with 0
offer_logic_check['previous_offer_still_valid'] = 'no overlap'
# Populate checking column with 1 if new offer overlaps with previous offer
offer_logic_check.loc[(offer_logic_check['shift_person'] == offer_logic_check['person']) & (offer_logic_check['shift_time'] > offer_logic_check['time']),'previous_offer_still_valid'] = 'overlap'
# Count occurences where new offer overlaps with previous offer
offer_logic_check['previous_offer_still_valid'].value_counts()

#### Can a person receive a new offer whilst still being aware of an active, not completed offer? >> Yes, seen upon visual inspection of the data
#### If yes, can this be the *same* offer? >> Yes, seen upon visual inspection of the data

### Step 3: Wrangle master table

#### Step 3.1 Identify unique blocks of offer receive events with views and completes

I need to be able to uniquely identify an "offer block". An offer block defines a person-x-offer combination, where an offer block starts with a receive and contains *exactly* one receive, and a maximum of one view and one complete. I will achieve this in two stages:
* In a first step, I will assign an ID to each offer block that uniquely identifies a person x offer combination
* Following this, I will split up offer blocks that still contain multiple receives because a person received a specific offer multiple times.

Finally, I will do some data checks

##### Step 3.1.1

In [ ]:
# Sort the master dataframe. This is essential for the further steps.
master = master.sort_values(by=['person', 'event_offer_id', 'time'])

In [ ]:
# Create a dataframe that creates a unique identifier for each person and offer combination
person_offer_id_df = master.loc[master['event'] == 'offer received', ['person', 'event_offer_id']].drop_duplicates().reset_index().rename(columns={'index':'person_offer_id'})
# Merge the id back into the master dataframe
master = pd.merge(master, person_offer_id_df, how = 'left', left_on = ['person','event_offer_id'], right_on = ['person','event_offer_id'])
# Note: This only affect event: receive, view and complete, because transactions do not have an event_offer_id. This will be tackled later-on.

In [ ]:
# Because one person can receive the same offer multiple times, some offer blocks will have multiple offer received. 
# The most times one person has received the same offer is:
max_same_offer = master.loc[master['event'] == 'offer received'].groupby(by=['person_offer_id']).size().max()
print("The most times one person has received the same offer is:", max_same_offer)

##### Step 3.1.2

Because one person can receive the same offer multiple times, these offer blocks need to be broken down further into offer blocks, where each "receive" starts its own offer block. First, I need to identify the person_offer_ids where the block has multiple receives.

In [ ]:
receives_per_block = master.loc[master['event'] == 'offer received'].groupby(by=['person_offer_id']).size()

In [ ]:
receives_per_block_df = pd.DataFrame(receives_per_block)

In [ ]:
blocks_need_subsplit = receives_per_block_df.loc[receives_per_block_df[0] > 1].index.tolist()

Now I will loop through these blocks and split them up into separate blocks.
Approach:
* Create a subset of the master dataframe that only contains records that belong to the specific person_offer_id
* Identify indices of "offer receive" records.
* Loop through each "offer receive" record. Allocate views and completes to the receive record. Mark receive, view and complete records that belong together with the same number in the 'sub_block' column.
* The allocation logic is:
> * Begin the loop at the earliest "receive".
> * In each loop, allocate a maximum of one view and one complete to the receive.
> * The views and completes get allocated to the earliest receive in whose validity time-frame they fall, unless that receive already has a view and complete allocated.
> * If a view or complete is "taken" by a receive, it cannot be re-allocated to another receive.
> * A view and a complete can only ever be allocated to one receive.


In [ ]:
# Establish list to store block allocations
sub_block_list = []

In [ ]:
for person_offer_id in blocks_need_subsplit: 
    # create the dataframe-subset of interest
    df = master.loc[master['person_offer_id'] == person_offer_id][['event','offer_valid_until','time']]
    df['sub_block'] = np.nan
    # Create a dictionary to store the indices, valid_untils and timestamps (i.e. time of reception) of receive records
    receive_dict = df.loc[df['event'] == 'offer received'][['offer_valid_until','time']].reset_index().to_dict(orient = 'records')
  
    # Loop through the "receive" records and allocate views and completes
    for i in range(len(receive_dict)):
        # set key variables for filtering
        offer_start_time = receive_dict[i]['time']
        validity = receive_dict[i]['offer_valid_until']
        # filter the block for views and receives within the right timeframe (from offer receive to offer validity end). Only keep such views and completes that do not yet belong to another receive
        df_for_loop = df.loc[(df['time'] <= validity) & (df['time'] >= offer_start_time) & (df['event'] != 'offer received') & (df['sub_block'].isna())]
        # reset the index so you can access it later
        df_for_loop = df_for_loop.reset_index()
        # number the rows using sql and select only the first entries
        df_from_loop = sqldf("SELECT * FROM (SELECT *, row_number() over (partition by event order by time) as counter_check_inner2 FROM df_for_loop) as subtable WHERE counter_check_inner2 = 1")
        
        #??????# Add a step where you only keep the "view" as belonging to the receive when it precedes the complete? It is currently unclear if s.o. can view after completion
        
        # extract the indices of the rows that belong to the "receive" you are examining in this loop
        rows_affected_list = df_from_loop['index'].to_list()

        # mark the "receive" you are assessing in the original data, so it is clear it belongs to a specific offer block
        df.loc[df.index == receive_dict[i]['index'], 'sub_block'] = i 
        # now mark the "view" and "complete" rows in the original df, so it is clear they belong to this specific "receive"
        df.loc[df.index.isin(rows_affected_list), 'sub_block'] = i 
        #print(df)
           
    # Create a dictionary with index and block allocation
    df = df.reset_index()
    allocation = df[['index','sub_block']].to_dict(orient = 'records')
    # Append this dictionary to the sub_block_list
    sub_block_list.append(allocation)
        

In [ ]:
# Flattening the resulting list into a list of dicts
# https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
# Question by Emma: https://stackoverflow.com/users/110527/emma
# Answer by Alex Martelli: https://stackoverflow.com/users/95810/alex-martelli
flat_list = [i for i in sub_block_list for i in i]

In [ ]:
# Filling NaNs in flat list with 9999, so I can delete them lateron. 
# These NaN values are "views" of expired offers and are hence irrelevant for offer awareness. They can be deleted.
df_flat_list = pd.DataFrame(flat_list)
df_flat_list.loc[df_flat_list['sub_block'].isna(), 'sub_block'] = 9999

### ----------------------   Start of investigation   ----------------------------------------------------------------------------------

In [ ]:
# Inspecting my flat list, because it has NaN values
df_flat_list = pd.DataFrame(flat_list)
df_flat_list.loc[df_flat_list['sub_block'].isna()]

In [ ]:
# Understanding why there are nans in my flat list

In [ ]:
# finding person_offer_id of an example case
master.loc[41162]

In [ ]:
m2x.loc[m2['person_offer_id'] == 191][['person','event','time','offer_duration','event_offer_id','offer_valid_until','sub_block']]

In [ ]:
# checking I created my block correctly >> I did
m2.loc[(m2['person'] == '1c587de019994f93a242c6864fd7bc55') & (m2['event_offer_id'] == 'ae264e3637204a6fb9bb56bc8210ddfd')]

In [ ]:
# Are all of these NaN cases offer views? >> Yes
df_flat_list_nans = df_flat_list.loc[df_flat_list['sub_block'].isna()]
df_flat_list_nans_checkframe = pd.merge(df_flat_list_nans, m2, how = 'left', left_on = 'index', right_index = True)

In [ ]:
df_flat_list_nans_checkframe['event'].value_counts()

Result: There can be offer views when the offer is already expired. That makes me believe that there can also be offer views when the offer has been completed - but that is just a guess.
The good thing about those offer views is, that I can just ignore them. They do not influence awareness. Therefore, I mark them as 9999 in the flat list before merging.

Further thoughts: At the moment, I allocate a view to the first receive, even if that receive is already completed. That could mean that a second offer does not receive the view, but that for the second offer the view would have created awareness. This means that I could "miss" periods of awareness. That would mean that i attribute too much spend to the unaware state. However, it is unclear and not clarified anywhere if someone can view an offer after having completed it. We do know from the data that people can see expired offers. Therefore, I just have to make a judgement call, and it is easier atm to just stick with allocating the view to the first receive, as my code already does that atm. There is no right or wrong.
### -------------------------------------   End of investigation  ----------------------------------------------------------------------------

### Continue

In [ ]:
# Merge the sub-block labels into master_interim, which is a temporary copy of master
master_interim = pd.merge(master, df_flat_list, how = 'left', left_index = True, right_on = 'index')
# Re-add indices from master to master_interim
master_interim.index = master.index
# reassign "master"
master = master_interim

In [ ]:
# Clean out columns with obsolete views
master = master.loc[master['sub_block'] != 9999]

In [ ]:
# Create a new block index from the original person_offer_id and the sub_blocks that uniquely identifies all sub-blocks
block_id_lookup = master[['person_offer_id','sub_block']].drop_duplicates().reset_index(drop=True)
block_id_lookup = block_id_lookup.reset_index().rename(columns={'index':'block_id'})

# Set a dummy value for NaN rows (the transaction rows) >> Not needed as no transactions in df anymore
# block_id_lookup.loc[(block_id_lookup['person_offer_id'].isna()) & (block_id_lookup['sub_block'].isna()), 'block_id'] = 99999

# Merge the new index into the dataframe
master = pd.merge(master, block_id_lookup, how = 'left', left_on = ['person_offer_id','sub_block'], right_on = ['person_offer_id','sub_block'])

# Remove obsolete indices
master = master.drop(['person_offer_id', 'sub_block','index'], axis=1)

##### Data checks:

In [ ]:
# Datacheck 1: No more than 3 rows per block_id (except transactions)
check1 = master.groupby(by=['block_id']).size().sort_values()

check1 = pd.DataFrame(check1)

check1.groupby(by=[0]).size()

In [ ]:
# Datacheck 2: Only 0 or 1 of each event type per block_id (except transactions)
check2 = master[['block_id','event']]

check2 = check2.pivot_table(index='block_id', columns = 'event',aggfunc=len, fill_value=0)

check2.max()

In [ ]:
# Datacheck 3: Every block has a receive

In [ ]:
check2.sum()

In [ ]:
check2.shape[0]

##### in-between step so I don't have to rerun code every time

In [ ]:
# Save the file
master.to_csv(r'2021_12_08_interim_file.csv')

In [16]:
# Read in the file
master = pd.read_csv(r'2021_12_08_interim_file.csv')

In the next step, I need to transform the dataframe so that each sub-block only has one row

#### Step 3.2 transform the table so that each block has one row

In [17]:
# Creating pivot with time of event
master_pivot = master.pivot_table(index='block_id', columns = 'event', values = 'time', fill_value=np.nan)

In [18]:
# Keep only one row with all relevant values per offer (the max statement does that)
master_collapse = master[['block_id','person','event_offer_id','event_reward','offer_type','offer_reward','offer_duration','offer_valid_until','offer_difficulty','email','web','social','mobile']]
master_collapse = master_collapse.groupby(by=['block_id']).max()

In [19]:
# Merge the collapsed master frame with the event - time information from the master_pivot
master = pd.merge(master_pivot, master_collapse, how = 'left', right_index = True, left_index = True)

In [20]:
master.head(10)

,offer completed,offer received,offer viewed,person,event_offer_id,event_reward,offer_type,offer_reward,offer_duration,offer_valid_until,offer_difficulty,email,web,social,mobile
block_id,,,,,,,,,,,,,,,
0,576.0,576,NaN,0009655768c64bdeb2e877511632db8f,2906b810c7d4411798c6938adc9daaa5,2.0,discount,2,168,744.0,10,1,1,0,1
1,NaN,336,372.0,0009655768c64bdeb2e877511632db8f,3f207df678b143eea3cee63160fa8bed,NaN,informational,0,96,432.0,0,1,1,0,1
2,NaN,168,192.0,0009655768c64bdeb2e877511632db8f,5a8bc65990b245e5a138643cd4eb9837,NaN,informational,0,72,240.0,0,1,0,1,1
3,414.0,408,456.0,0009655768c64bdeb2e877511632db8f,f19421c1d4aa40978ebb69ca19b0e20d,5.0,bogo,5,120,528.0,5,1,1,1,1
4,528.0,504,540.0,0009655768c64bdeb2e877511632db8f,fafdcd668e3743c1bb461111dcafc2a4,2.0,discount,2,240,744.0,10,1,1,1,1
5,NaN,168,216.0,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,NaN,bogo,5,120,288.0,5,1,1,1,1
6,NaN,576,630.0,00116118485d4dfda04fdbaba9a87b5c,f19421c1d4aa40978ebb69ca19b0e20d,NaN,bogo,5,120,696.0,5,1,1,1,1
7,576.0,408,432.0,0011e0d4e6b944f998e987f904e8c1e5,0b1e1539f2cc45b7b9fa7c272da2e1d7,5.0,discount,5,240,648.0,20,1,1,0,0
8,252.0,168,186.0,0011e0d4e6b944f998e987f904e8c1e5,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0,discount,3,168,336.0,7,1,1,1,1


In [21]:
master = master[['event_offer_id','person','offer received','offer viewed','offer completed','event_reward','offer_type','offer_reward','offer_duration','offer_valid_until','offer_difficulty','email','web','social','mobile']]

### Step 4: Create derived variables

In [22]:
# was valid until (either expiry date or completion date)
master['factual_validity_end'] = master[['offer completed','offer_valid_until']].min(axis = 1)

# was completed
master['was_completed'] = 1
master.loc[master['offer completed'].isna(),'was_completed'] = 0

# was aware binary
master['was_aware'] = 0
master.loc[(master['offer viewed'] != np.nan) & (master['offer viewed'] < master['factual_validity_end']), 'was_aware'] = 1

# time taken to become aware
master['time_lag_awareness'] = np.nan
master.loc[master['was_aware'] == 1, 'time_lag_awareness'] = master['offer viewed'] - master['offer received']

# time taken to complete when aware
master['time_lag_completion'] = np.nan
master.loc[(master['was_aware'] == 1) & (master['was_completed'] == 1), 'time_lag_completion'] = master['offer completed'] - master['offer viewed']

# time left between completion and natural expiry when aware
master['time_lag_completion_to_expiry'] = np.nan
master.loc[(master['was_aware'] == 1) & (master['was_completed'] == 1), 'time_lag_completion_to_expiry'] = master['offer_valid_until'] - master['offer completed'] 

The awareness period for offers with offer awareness is 'offer viewed' to 'factual_validity_end'

### Step 5: Insert transaction values

#### 5.1: Set up transaction df

In [23]:
# Set up transaction dataframe
tx = transcript.loc[transcript['event'] == 'transaction'].reset_index().rename(columns={'index':'tx_id','time':'time_of_transaction'})
tx = pd.concat([tx, pd.DataFrame(tx['value'].tolist())], axis=1) # Extract the amount value
tx = tx.drop(['event','value'], axis = 1)

In [24]:
tx.head()

,tx_id,person,time_of_transaction,amount
0,12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
1,12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
2,12659,54890f68699049c2a04d415abc25e717,0,13.23
3,12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
4,12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


#### 5.2 Set up awareness dataframe

Each transaction needs to be allocated to one or more block ids. To achieve this, the awareness dataframe is created that contains one row for each point in time t that a person was aware of an offer. Then this is merged onto the transaction dataframe from step 5.1 from this df one can then see if a transaction happened during aware or anaware time and if it was during unaware time if the person was only aware of one offer or of multiple offers.
If a person was aware of multiple offers, the transaction gets allocated based on the following logic:
* The transaction gets allocated to the first offer that was completed because then the spend clearly helped to complete that offer
* If no offer was completed then the amount is split evenly
* More complex scenarios are possible e.g. that with one transaction a person completed two offers at the same time However, such special cases are not considered in the data wrangling as their number is estimated to be few.

In [25]:
# Set up the awareness frame - only keep offers that a person was aware of. All other time is unaware time.
aw = master[['person','event_offer_id','offer viewed','factual_validity_end','was_aware']].loc[master['was_aware']==1].reset_index().drop('was_aware', axis=1)

In [26]:
aw['rows_needed'] = aw['factual_validity_end'] - aw['offer viewed'] + 1

In [27]:
# Create a dictionary with block ids and number of rows needed per block id
block_ids, lines = aw['block_id'],aw['rows_needed']
dict1 = pd.Series(lines.values, block_ids.values).to_dict()

In [28]:
# Create list of all block ids to use in list comprehension
block_id_list = aw['block_id'].tolist()

In [29]:
# Create list with number of each block id appearing as many times as rows are needed
# Example: block id 1 had awareness for 10 hours >> For block id 1, 10 rows are needed >> block id 1 appears 10 times in the list
row_list = [[i for j in range(int(dict1[i]))] for i in block_id_list]

# Flatten the list as is currently list of lists
# Source Stackoverflow: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-a-list-of-lists
# Question by Emma: https://stackoverflow.com/users/110527/emma
# Answer by Alex Martelli: https://stackoverflow.com/users/95810/alex-martelli
row_list = [i for sublist in row_list for i in sublist]

In [30]:
# Merge in the original information 
aw_expanded = pd.merge(pd.DataFrame(row_list).rename(columns={0:'block_id'}), aw, how = 'left', left_on = 'block_id', right_on = 'block_id')

In [31]:
# Use SQL to add row numbers that can then be transformed to hours
aw_expanded = sqldf("SELECT *, row_number() over (partition by block_id) as counter FROM aw_expanded")

In [32]:
# Calculate columns with separate hours
aw_hourly = aw_expanded
aw_hourly['hour'] = aw_hourly['counter'] + aw_hourly['offer viewed'] - 1

In [33]:
aw_hourly = aw_hourly[['block_id','person','event_offer_id','hour','offer viewed']]

#### 5.3 Combine both dataframes, create control and treatment frame and allocate block ids to transactions where a customer was aware ( = treatment)

In [35]:
tx_block_ids = pd.merge(tx, aw_hourly, how = 'left', left_on = ['person','time_of_transaction'], right_on =['person','hour'])

In [36]:
# What is the maximum how many blocks a transaction could be allocated to
count_tx_ids = tx_block_ids[['tx_id']].groupby(by=['tx_id']).size()
count_tx_ids.max()

3

In [37]:
# How many overlap cases exist?
pd.DataFrame(count_tx_ids).value_counts()

1    133710
2      5117
3       126
dtype: int64

##### Separate the transactions that happened during non-awareness (control) from those that happened during awareness

In [38]:
# Split tx_block_ids into a control and treatment frameb
tx_control = tx_block_ids.loc[tx_block_ids['event_offer_id'].isna()] # Not aware or no active offer
tx_treatment = tx_block_ids.loc[~tx_block_ids['event_offer_id'].isna()]

##### Separate the non-overlap treatment cases from the overlap treatment cases

In [39]:
# identify transaction ids with multiple block_ids, i.e. those with more than 1 row in the tx_treatment dataframe
tx_treatment_gr = tx_treatment[['tx_id']].groupby(by=['tx_id']).size()
tx_treatment_block_id_count = pd.merge(tx_treatment, pd.DataFrame(tx_treatment_gr).rename(columns={0:'count'}), how = 'left', left_on = 'tx_id', right_index=True)

In [40]:
# Remove cases without overlap and store in separate df as these need further treatment
tx_treatment_no_overlap = tx_treatment_block_id_count.loc[tx_treatment_block_id_count['count'] == 1]
tx_treatment_overlap = tx_treatment_block_id_count.loc[tx_treatment_block_id_count['count'] > 1]

In [41]:
# Create a dictionary for the non-overlap treatment cases where transaction id is the key and block id is the value
id_tx_fordict, block_id_fordict = tx_treatment_no_overlap['tx_id'],tx_treatment_no_overlap['block_id']
dict_treatment_no_overlap = pd.Series(block_id_fordict.values, id_tx_fordict.values).to_dict()

##### Allocate transaction ids with offer overlap to the correct offer(s) (i.e. block_ids)

In [43]:
# Add information on whether an offer was completed
tx_treatment_overlap = pd.merge(tx_treatment_overlap, master[['was_completed','offer completed']], how = 'left', left_on = 'block_id', right_index = True)

In [46]:
# Source: dfriends.com by Dan Friedman, April 15, 2019: https://dfrieds.com/data-analysis/rank-method-python-pandas.html
# Add ranks to all completed block_ids and give those that are completed simultaneously the same rank
tx_treatment_overlap['rank'] = tx_treatment_overlap.groupby(['tx_id'])['offer completed'].rank(method = 'min')

In [47]:
# Sort dataframe within tx id by time the offer was viewed
tx_treatment_overlap = tx_treatment_overlap.sort_values(by=['tx_id','rank'])

In [49]:
# Create list of all transaction ids that need to be processed due to offer overlap
tx_ids_with_overlap = tx_treatment_overlap['tx_id'].tolist()
# Remove all duplicates
tx_ids_with_overlap = list(set(tx_ids_with_overlap))

In [77]:
# Allocate block ids to transaction ids; in the dict, the key will be the transaction id and the value the block id
# If a transaction needs to be split between multiple offers, then there will be a list of offer ids

dict_treatment_overlap = {}
tx_treatment_overlap_reduced = tx_treatment_overlap[['tx_id','block_id','was_completed','rank']]
for tx_id in tx_ids_with_overlap:
    # Reduce dataframe to dataframe of interest
    df = tx_treatment_overlap_reduced.loc[tx_treatment_overlap_reduced['tx_id'] == tx_id]
    sum_completed = df['was_completed'].sum()
    if sum_completed == 0:
        dict_treatment_overlap[tx_id] = df['block_id'].tolist()
    elif sum_completed > 0:
        dict_treatment_overlap[tx_id] = df['block_id'].loc[df['rank'] == 1].tolist()

#### 5.4 Create a final awareness (=treatment) dataframe with one row per transaction id and block id as well as the correct amount to be allocated to the block id

In [99]:
# Create a dataframe that allocates transactions to block ids for those with and without awareness and those with and without overlap
# https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe
block_id_allocation = pd.DataFrame(dict_treatment_overlap.items()).set_index(0).rename(columns={1:0}).append(pd.DataFrame.from_dict(dict_treatment_no_overlap, orient = 'index'))
block_id_allocation = block_id_allocation.reset_index().rename(columns = {'index':'tx_id'})

In [101]:
# Data check for duplicates; needs to be 1
print(block_id_allocation.groupby(by='tx_id').size().max())

1


In [102]:
# merge into tx dataframe
block_id_allocation = pd.merge(tx, block_id_allocation, how = 'left', left_on = 'tx_id', right_on = 'tx_id').rename(columns = {0:'block_id'})

In [104]:
# Get number of block_ids that a tx_id is allocated to to reduce the amount
dict_treatment_overlap_len = {}
for key, value in dict_treatment_overlap.items():
    try:
        a = len(value)
    except:
        a = 1
    dict_treatment_overlap_len[key] = a

block_id_allocation = pd.merge(block_id_allocation,pd.DataFrame.from_dict(dict_treatment_overlap_len, orient = 'index'), how = 'left', left_on = 'tx_id', right_index = True).rename(columns={0:'divider'})
block_id_allocation.loc[block_id_allocation['divider'].isna(), 'divider'] = 1

# Calculate amount to be allocated to each separate block_id
block_id_allocation['corrected_amount'] = block_id_allocation['amount']/block_id_allocation['divider']

In [105]:
# Expand the df to have multiple rows where there are lists with multiple block_ids for a tx_id
# https://stackoverflow.com/questions/38203352/expand-pandas-dataframe-column-into-multiple-rows
block_id_allocation = block_id_allocation.explode('block_id')

#### 5.5 Aggregate the transactions by block_id and join into the master table

In [107]:
amount_per_block_id = block_id_allocation. groupby(['block_id'])['corrected_amount'].sum()

In [ ]:
aw_final = pd.merge(master, amount_per_block_id, how = 'left', left_on = 'block_id', right_on = 'block_id')

In [ ]:
# Data Checks:
# Check 1: Amounts are only allocated to aware offers
aw_final['was_aware'].loc[~aw_final['corrected_amount'].isna()].min() # >> Should be 1

In [125]:
# Check 2: All completed offers where someone was aware have an amount
aw_final.loc[(aw_final['was_completed'] == 1) & (aw_final['corrected_amount'].isna()) & (aw_final['was_aware'] == 1)] # >> Should return no results

,event_offer_id,person,offer received,offer viewed,offer completed,event_reward,offer_type,offer_reward,offer_duration,offer_valid_until,...,web,social,mobile,factual_validity_end,was_completed,was_aware,time_lag_awareness,time_lag_completion,time_lag_completion_to_expiry,corrected_amount
block_id,,,,,,,,,,,,,,,,,,,,,


In [126]:
# Remove all unaware block_ids from the final awareness dataframe again after wrangling has passed checks
aw_final = aw_final.loc[aw_final['was_aware'] == 1]

### Step 6: Create hourly average spend
Daily averages for the treatment group are on person - offer level.
Daily averages for the control group are per person, counting all unaware time.

#### Step 6.1: Treatment group

In [134]:
# add duration of awareness by hour to aw_final
aw_final = pd.merge(aw_final,aw[['block_id','rows_needed']], how = 'left', left_index = True, right_on = 'block_id')

In [137]:
treatment_hourly_averages = aw_final.groupby(by=['person', 'event_offer_id']).agg({'corrected_amount':sum, 'rows_needed':sum})

In [139]:
# CAREFUL: I currently have too many rows because validity can exceed experiment timeframe
treatment_hourly_averages['hourly_avg'] = treatment_hourly_averages['corrected_amount']/treatment_hourly_averages['rows_needed']

#### Step 6.2: Control Group 

In [152]:
# Total hours of experiment
max_hour = transcript['time'].max()

In [156]:
# https://stackoverflow.com/questions/41415017/count-unique-values-using-pandas-groupby/41415028
person_aware_time = aw_hourly.loc[aw_hourly['hour'] <= max_hour].groupby('person')['hour'].nunique()

In [164]:
person_unaware_time = (pd.DataFrame(person_aware_time)['hour'] - max_hour) * (-1)

In [172]:
# Time every person was unaware/had no active offer
control_hourly_averages = pd.merge(pd.DataFrame(person_unaware_time), pd.DataFrame(tx_control.groupby('person')['amount'].sum()), left_index = True, right_index = True)

In [174]:
control_hourly_averages['hourly_avg'] = control_hourly_averages['amount']/control_hourly_averages['hour']

In [177]:
# Data check
control_hourly_averages.isna().sum()

hour          0
amount        0
hourly_avg    0
dtype: int64

  
Ideas for further variables I could construct and use for prediction are:

    * Was previously aware of offer
    * Previously received purely informational offer
    * Time since last offer awareness
    * Time since last offer completion



# NEXT STEPS:
Important: Need to cut the awareness hour timeframes to max 714! (aw_hourly).
I think that is somewhere very much in the beginning, where I calculate the maximum offer validity. There I should set a max.

Create avg daily spend per
- person in control group
- block_id in awareness (treatment) df

### Next stage:
Check distribution of key variables between those with and without demographic information, and drop them if no massive differences.

### Next stage: 
Show some descriptive statistics to understand your data. specifically average spend charts and offer completion rates. For average spend, also look at those who complete versus do not complete versus control, because I want to know if simple awareness already changes spend.Consider how to handle the purely informational offers.

### Next big stage:
Run t-tests or ANOVAS or even a GLM to assess success of each offer. measure success as avg spend per hour compared to non-aware times. Your separate datapoints are person-x-offerX and person-x-unaware_time, where each shows average hourly spend.

Then look at offer completion rates when aware, i.e. conversion. Do ANOVA with comparsion between offers (or whatever the equivalent is, because this will be binary data) if you feel like it.

### Next big stage:
Select best offer. Reduce sample to people who have received that offer. Predict either offer conversion (i.e. uptake/completion) or predict increase in spend. If spend only really increases for people who complete an offer, run prediction on completion only. If spend also increases when people are just aware, then predict spend.


# Merging profile table
master = pd.merge(master, profile[['gender','age','became_member_on','membership_duration','income']], how = 'left', left_on = master['person'], right_on = profile['id'])
master = master.drop('key_0', axis = 1)